## Evaluación diagnostica gitflow

Programar en Python cuatro funciones:

1. Los top 10 tweets más retweeted.
2. Los top 10 usuarios en función a la cantidad de tweets que emitieron.
3. Los top 10 días donde hay más tweets.
4. Top 10 hashtags más usados.
Dataset: https://www.kaggle.com/datasets/prathamsharma123/farmers-protest-tweets-dataset-raw-json

In [1]:
# Import
import json

### Función top retweeted